In [3]:
import tensorflow as tf
from nnet_for_hist_dropout_stride import *
import logging

In [5]:
config = Config()

In [9]:
predict_year = 2016

In [10]:
saver=tf.train.Saver()
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.22)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
sess.run(tf.initialize_all_variables())
saver.restore(sess, config.save_path+str(predict_year)+"CNN_model.ckpt")

INFO:tensorflow:Restoring parameters from /home/neelansh/Desktop/Crop/TRAIN_RESULTS/2016CNN_model.ckpt


In [11]:
filename = 'histogram_all_full' + '.npz'
content = np.load(config.load_path + filename)
image_all = content['output_image']
yield_all = content['output_yield']
year_all = content['output_year']
locations_all = content['output_locations']
index_all = content['output_index']

In [12]:
index_validate = np.nonzero(year_all == predict_year)[0]

In [13]:
image_validate=image_all[index_validate]

In [30]:
print(image_validate.shape)

(1363, 32, 32, 9)


In [14]:
yield_validate=yield_all[index_validate]

In [16]:
pred = []
real = []

In [17]:
for j in range(int(image_validate.shape[0] / config.B)):
    real_temp = yield_validate[j * config.B:(j + 1) * config.B]
    pred_temp= sess.run(model.logits, feed_dict={
        model.x: image_validate[j * config.B:(j + 1) * config.B,:,0:config.H,:],
        model.y: yield_validate[j * config.B:(j + 1) * config.B],
        model.keep_prob: 1
    })
    pred.append(pred_temp)
    real.append(real_temp)

In [42]:
pred_final = (lambda pred: [item for sublist in pred for item in sublist])(pred)

In [44]:
real_final = (lambda real: [item for sublist in real for item in sublist])(real)

In [21]:
print(pred)

[array([141.72972 ,  82.629555, 136.73187 , 185.89613 , 127.69431 ,
       161.0809  , 190.46535 , 124.60304 , 128.30031 , 140.96977 ,
       111.67516 , 121.87965 , 102.63128 , 131.41057 , 149.41573 ,
       140.45094 , 120.47035 , 121.01942 , 136.03026 , 126.34319 ,
       112.58503 , 130.3218  ,  92.76379 , 110.09421 , 103.50918 ,
       128.63277 , 143.71877 , 124.5572  , 182.67699 , 165.88885 ,
       110.211   , 222.06058 ], dtype=float32), array([165.4209  , 157.19008 , 151.4864  , 152.61113 , 163.2902  ,
       154.09782 , 142.47801 , 164.81392 , 127.461075, 136.34465 ,
       137.45625 , 146.3953  , 162.64964 , 117.14596 , 149.3239  ,
       154.71297 , 165.3527  , 166.42595 , 141.60081 , 156.01604 ,
       101.985435, 114.52273 , 102.7051  , 118.867714, 110.85093 ,
       111.29371 ,  68.161   ,  84.093475,  68.138756, 102.928154,
        69.335625,  90.55899 ], dtype=float32), array([151.56752 , 134.96776 ,  67.05396 ,  61.715702,  90.8196  ,
        66.55708 ,  91.638596, 1